# Diagonalización Por Método De Gauss, Factorización LU y Factorización QR

In [1]:
import numpy as np
import sympy as sm

## Ejercicio 1
En este ejercicio deberán implementar el método de eliminación Gaussiana para diagonalizar una matriz cuadrada.

**Paso 1**: Programa una función que reciba una Matriz cuadrada de tamaño $n×n, M$, y que por medio del método de eliminacion Gaussiana regrese una **Matriz triangular superior**. Una vez que hayas terminado prueba tu función con distintas matrices cuadradas usando el comando.

**¡¡Cuidado!!** Hay que tener cuidado cuando hay un 0 en la diagonal. Lo más fácil es suponer que eso no va a pasar y programar el método de eliminación Gaussiana omitiendo ese hecho; pero lo correcto es corroborar que eso no pase llevando a cabo un intercambio con alguno de los renglones debajo del renglón que tiene 0 en la diagonal. Intenta resolver el primer caso y luego agregarle el paso en el que te aseguras que no haya ceros en la diagonal.

Problema a considerar.
\begin{equation}
A = \begin{bmatrix}
1 & 2 & -2 & 1 \\
4 & 5 & -7 & 6\\
5 & 25 & -15 & -3\\
6 & -12 & -6 & 22
\end{bmatrix}
\end{equation}

\begin{equation}
A \sim \begin{bmatrix} 1 & 2 & -2 & 1\\
0 & -3 & 1 & 2\\
0 & 15 & -5 & -8\\
0 & -24 & 6 & 16 \\
\end{bmatrix}
\sim \begin{bmatrix}
1 & 2 & -2 & 1\\
0 & -3 & 1 & 2\\
0 & 0 & 0 & 2\\
0 & 0 & -2 & 0 \
\end{bmatrix} \end{equation}

![gauss.png](attachment:gauss.png)

1. Se busca en la columna j de la matriz el mayor número en valor absoluto y dicho valor se ubica.
2. Se pasa intercambian renglones de la matriz y se procede a hacer la eliminación Gaussiana.

In [2]:
def cambio_filas(M,b): #Función que se aplicara cuando haya un 0 en la primera entrada
  m1=len(M)#Número de filas (Al ser una matriz cuadrada es tambien el número de columnas)
  if M[0][0]==0:#Si la entrada 1,1 es 0
    for i in range (1,m1): #Se revisaran las demas filas
      if M[i][0]!=0: #Si la entrada i,1 es diferente de 0
        aux=M[0]
        M.insert(0,M[i])
        M.pop(1)
        M.insert(i,aux)
        M.pop(i+1) #Se realiza el cambio de filas en la Matriz
        aux2=b[0]
        b.insert(0,b[i])
        b.pop(1)
        b.insert(i,aux2)
        b.pop(i+1) #Tambien se realiza este cambio en el vector de resultados
        break #Ahi rompemos el if para que no se repita (pues no queremos que se vuelva a cambiar la fila)
    return M,b
  else:
    return M,b


In [3]:
def Fila_normal(M,b): #Función que normalizara la fila 1 de mi Matriz
  M,b=cambio_filas(M,b) #Primero me aseguro que en la entrada 1,1 no haya 0, usando mi función previa de cambio de filas
  X=M[0][0] #Fijo el valor 1,1
  if X==0:
    X=1
  for i in range(len(M[0])): #Hago que en mi primera fila todos los elementos sean divididos entre en valor 1,1 original
    M[0][i]=M[0][i]/X
  b[0][0]=b[0][0]/X #Hago lo mismo con el primer valor en mi vector de resultados
  return M,b

In [4]:
def Columna_0(M,b): #Función que hara que la columna 1 de mi matriz solo tenga un 1 en la entrada 1,1
  M,b=Fila_normal(M,b) #Uso mi función previa para normalizar la fila 1
  for i in range (1,len(M)): #En cada fila de mi matriz
    X=M[i][0] #Fijo el valor original de mi primer elemento de cada fila
    if M[i][0]==0: #Si este es 0, la fila ya esta como quiero entonces pasare a la siguiente
      pass
    else:
      for j in range (len(M[i])): #En cada elemento de mi renglon i
        M[i][j]=M[i][j]-(M[0][j]*X) #la entrada i,j (i+1,j+1 si empiezas con 1,1 en ves de 0,0) el elemento sera si mismo
        #menos la entrada 0,j por el valor original de i,0, esto hara que la entrada i,0 sea 0 (pues la entrada 0,j es 1 por mi función normalizadora)
        #y el valor de mi fila no cambie
      b[i][0]=b[i][0]-(X*b[0][0]) #Por ultimo aplico esas mismas operaciones a mi elemento de mi vector de resultados
  return M,b

In [5]:
def reduc_matr(M,b):#Función que me reducira la matriz n*n a una (n-1)*(n-1)
    A = [] #FIjo una lista vacia
    for i in range(1, len(M)):#A partir de la segunda fila de mi matriz original
        A.append((M[i][1:]).copy()) #Agregare a mi lista A los elementos de mis filas a partir del segundo
    b2=[]
    for j in range (1,len(b)): #Tambien hago esta reducción tomando todos los elementos de mi vector resultado menos el primero
      b2.append(b[j])
    return A,b2 #Regreso A

In [6]:
def introducir_matriz(M,A,b,b2):#Ahora para agregar una matriz menor a una mayor (En especifico la matriz creada con la función anterior)
  for i in range(1,len(M)):
    M[i][1:]=A[i-1] #La entrada de mi matriz a partir del segundo renglon y de la segunda columna sera cada entrada de mi matriz A
  for j in range (1,len(b)):
    b[i][0]=b2[i-1][0] #Hago lo mismo con mi elemento de mi vector de resultado
  return M,b

In [7]:
def metodo_Gauss(M,b): #Función que me hara triangular mi matriz
  M,b=Columna_0(M,b) #Primero hago la primera columna una columna con un 1 y 0s
  A=[M,b] #Lista donde agregare cada sub matriz necesaria con su respectiva columna con 0s
  for i in range(1,len(M)):#Debido a que en una matriz n*n, puedo sacar n-1 submatrices
    a,d=reduc_matr(A[2*(i-1)],A[(2*(i-1))+1]) #Saco mi respectiva sub matriz a cada sub matriz de M y b
    a,d=Columna_0(a,d) #Cada sub matriz hago que su columna 1 sea de 0s
    A.append(a)
    A.append(d)
    #Agrego a mi lista A las submatrices con su columna 0, haciendo que en los lugares pares
    #Esten mis submatrices de M y en mis impares mis submatrices de b
  for j in range ((len(M)-1)):
    A[-4],A[-2]=introducir_matriz(A[-4],A[-2],A[-3],A[-1])
    #Usando mi función de introducir sub matrices, las reemplazo de derecha a izquierda
    A.pop(-1)
    A.pop(-2)
    #Elimino las ultimas 2 submatrices pues estan ya fueron integradas a la siguiente matriz
  return A[0],A[1] #Al final mis unicos 2 elementos de mi lista A sera la matriz M triangulada y su respectivo vector de resultados

In [8]:
a,b=metodo_Gauss(np.random.random((10,10)),np.random.random((10,1)))

In [9]:
sm.Matrix(a)

Matrix([
[1.0, 0.417517203137176, 0.208604331406595, 0.108649482565153,  0.779761247981218, 0.214862454641656,  0.924801312943859,  0.967347665476015,   1.11490641462658,  0.283749293035101],
[  0,               1.0, 0.428133728722984,  0.83910388429456, -0.418155960660169,  0.28083994806341,  0.111639883928784, 0.0967511845896424,   1.01743345521539,   1.57643032294342],
[  0,                 0,               1.0, 0.840468693111536,  0.520465017465707,  1.19390396965316,   1.55462937759505,  0.831140504307203,  0.509487400898541,  0.839322167973449],
[  0,                 0,                 0,               1.0,   2.57924339853969,  1.27605980294397,    2.2640758179944,  -1.51021645730699,   -5.6247989519506,   -4.4284900514734],
[  0,                 0,                 0,                 0,                1.0, -8.24919881883957,  0.755385545407286,   -5.2051532403852,  -8.46455144458063, -0.968270746814579],
[  0,                 0,                 0,                 0,              

In [10]:
sm.Matrix(b)

Matrix([
[  1.08930039989504],
[ 0.735585032524642],
[  1.57384335774922],
[ -0.41518683325115],
[ -3.50268049602301],
[ 0.424470503230144],
[ 0.864704157047311],
[-0.278670426078764],
[ 0.374582087338386],
[ -0.65325949772945]])

**Paso 2:** Programa una función que reciba una Matriz cuadrada de tamaño $n×n, T$, que además sea triangular superior (como la que obtenemos al usar las función que programaste en el paso 1) y que se encargue de eliminar todos los elementos superiores a la diagonal usando una vez más el método de eliminación Gaussiana.

In [11]:
def Jordan(M,b): #Pido mi matriz de coeficientes y la de resultados
  for i in range (1,len(M)): #Hago un indice que vaya de 1 a la longitud de M-1 (ya que usare indice negativo)
    for j in range(0,len(M)-i):#Un indice que vaya desde 0 a M-i pues los renglones por debajo de mi diagonal no me importan
      X=M[j][-i] #FIjo mi coeficiente que quiero eliminar
      M[j][-i]=M[j][-i]-(X*M[-i][-i])#Hago que el coeficiente se elimine
      b[j][0]=b[j][0]-(X*b[-i][0]) #le aplico la misma operación al vector de resultados
  return M,b #Regreso M,b


In [12]:
a2,b2=Jordan(a,b)

In [13]:
sm.Matrix(a2)

Matrix([
[1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0]])

In [14]:
sm.Matrix(b2)

Matrix([
[-0.990574945416678],
[ -1.15555674248365],
[ -0.38657960332863],
[  2.78631814574388],
[  1.73875705200123],
[-0.814351271771536],
[-0.187387888532157],
[-0.432307762389658],
[  1.73670079307618],
[ -0.65325949772945]])

**Paso 3:** Finalmente, programa una función que reciba una Matriz cuadrada de tamaño n×n, M, y que mande a llamar las dos funciones que programaste anteriormente, de forma que al final regrese la diagonal de la matriz que obtuviste al final del procedimiento

In [15]:
def metodo_Gauss_Jordan(M,b):
  M,b=metodo_Gauss(M,b)
  M,b=Jordan(M,b)
  return M,b

In [16]:
a3,b3=metodo_Gauss_Jordan([[15,789,12],[13,68,14],[20,9,15]],[[15],[68],[21]])

In [17]:
sm.Matrix(a3)

Matrix([
[1.0,   0,   0],
[  0, 1.0,   0],
[  0,   0, 1.0]])

In [18]:
sm.Matrix(b3)
#Rectifique los resultados con calculadoras de sistemas de ecuaciones y son correctos

Matrix([
[  -8.68219466434815],
[-0.0134077701002151],
[   12.9843042145243]])

## Ejercicio 2:
En este ejercicio deberán implementar el método para encontrar la factorización $LU$ de una matriz cuadrada.\

**Paso 1:** Programen una función que calcule la factorización $LU$ de una matriz cuadrada, $A$, de tamaño $n×n$ y que regrese tanto la matriz triangular inferior $L$, como la matriz triangular superior $U$. Una vez que hayas terminado prueba tu función con distintas matrices cuadradas usando el comando python`` A = np.random.random((n,n)) ``con diferentes valores para $n$.
1. La matriz $L$ inicialmente es la matriz identidad con el mismo número de renglones de $A$. Si se utilizó la operación $R_i = R_i + c*Pivote_j$ entonces en la posición $L[i, j]$ se coloca $-c$.
2. La matriz $U$ es la matriz que queda en al escalonar $A$.
\begin{equation}
A = \begin{bmatrix}
2 & 1 & 1\\
4 & 1 & 0\\
-2 & 2 & 1 \\
\end{bmatrix}
\end{equation}
\begin{equation}
A \sim \begin{bmatrix}
2 & 1 & 1\\
0 & -1 & -2\\\
-2 & 2 & 2 \\
\end{bmatrix},
\quad E_{1} =
\begin{bmatrix}
1 & 0 & 0\\
-2 & 1 & 0\\
0 & 0 & 1 \\
\end{bmatrix},
\quad E_{1}^{-1} =
\begin{bmatrix}
1 & 0 & 0\\
2 & 1 & 0\\
0 & 0 & 1 \\
\end{bmatrix}
\end{equation}
\begin{equation}
A \sim \begin{bmatrix}
2 & 1 & 1\\
0 & -1 & -2\\
0 & 3 & 1 \\
\end{bmatrix},
\quad E_{2} =
\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
1 & 0 & 1\\\
\end{bmatrix},
E_{2}^{-1} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
-1 & 0 & 1\\
\end{bmatrix},
\quad E_{1}^{-1}E_{2}^{-1} =
\begin{bmatrix}
1 & 0 & 0\\
2 & 1 & 0\\
-1 & 0 & 1\\
\end{bmatrix}
\end{equation}
\begin{equation}
A \sim \begin{bmatrix}
2 & 1 & 1\\
0 & -1 & -2\\
0 & 0 & 4 \\
\end{bmatrix} = U,
\quad E_{3} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 3 & 1\\
\end{bmatrix},
E_{3}^{-1} = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & -3 & 1\\
\end{bmatrix},
\quad L = E_{1}^{-1}E_{2}^{-1}E_{3}^{-1} =
\begin{bmatrix} 1 & 0 & 0\\
2 & 1 & 0\\
-1 & -3 & 1 \\
\end{bmatrix}
\end{equation}

**Información:** Cuando no es posible factorizar una matriz de la forma $LU$, lo que se aplica es la factorización $P⋅A=L⋅U$, donde $P$ es una matriz de permutación.

**Paso 2:** Corrobora que el producto de matrices $LU$ regresa la matriz original $A$ a la que se le encontró la factorización.

In [19]:
import numpy as np

def elimina_ceros_primera_columna(U, L):
    """
    Elimina los ceros en la primera columna y ajusta las matrices U y L.
    """
    con = U[0][0]
    for i in range(1, len(U)):
        if U[i][0] != 0:
            c = -U[i][0] / con
            aux_2 = [c * k for k in U[0]]
            aux_3 = [U[i][l] + n for l, n in zip(range(len(U[i])), aux_2)]
            U[i] = aux_3
            L[i][0] = -c

def L_U(M, L):
    """
    Realiza la factorización LU de la matriz M.
    """
    U = M.copy()

    if U[0][0] == 0:
        # Intercambia las filas si el primer elemento de la primera fila es cero
        for i in range(1, len(U)):
            if U[i][0] != 0:
                aux = U[i].copy()
                f_u = U[0].copy()
                U[0], U[i] = aux, f_u
                break

    elimina_ceros_primera_columna(U, L)

    if len(U[0]) != 1:
        # Llama recursivamente para la submatriz U[1:, 1:] y la submatriz L[1:, 1:]
        auxiliar, L_aux = L_U(np.array(U)[1:, 1:].tolist(), np.array(L)[1:, 1:].tolist())
        auxiliar.insert(0, U[0])
        L_aux.insert(0, np.array(L)[:, 0:1].transpose().tolist()[0])
    else:
        auxiliar, L_aux = U, L

    # Agrega ceros para rellenar
    for i in range(1, len(auxiliar)):
        auxiliar[-i].insert(0, 0)
        L_aux[-i].insert(0, 0)

    return auxiliar, L_aux

def LU(Matriz):
    """
    Realiza la factorización LU y devuelve las matrices L y U.
    """
    id = np.identity(len(Matriz[0])).tolist()
    U, L_trans = L_U(Matriz, id)
    return np.array(L_trans).transpose().tolist(), U

# Ejemplo de uso
A = np.random.randint(100, size=(5, 5))
print('A:', A, sep='\n', end='\n\n\n')
L, U = LU(A)
L = np.array(L)
print('L:', L, sep='\n', end='\n\n\n')
U = np.array(U)
print('U:', U, sep='\n', end='\n\n\n')
print('LU:',L.dot(U), sep='\n')


A:
[[81 85 52 17 86]
 [ 0  1 51 95  2]
 [25  1 79  7  7]
 [65  7 17 33 17]
 [93  0 69 29 88]]


L:
[[  1.           0.           0.           0.           0.        ]
 [  0.           1.           0.           0.           0.        ]
 [  0.30864198 -25.           1.           0.           0.        ]
 [  0.80246914 -61.           2.30890052   1.           0.        ]
 [  1.14814815 -97.           3.70680628   1.27000543   1.        ]]


U:
[[ 8.10000000e+01  8.50000000e+01  5.20000000e+01  1.70000000e+01
   8.60000000e+01]
 [ 0.00000000e+00  1.00000000e+00  5.10000000e+01  9.50000000e+01
   2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.33700000e+03  2.37600000e+03
   3.10000000e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.28052356e+02
  -1.57591623e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   7.10904274e+01]]


LU:
[[81.         85.         52.         17.         86.        ]
 [ 0.          1.         51.         95.          2.       

In [20]:
A = np.random.randint(100, size=(5, 5))
print('A:', A, sep='\n', end='\n\n\n')
L, U = LU(A)
L = np.array(L)
print('L:', L, sep='\n', end='\n\n\n')
U = np.array(U)
print('U:', U, sep='\n', end='\n\n\n')
print('LU:',L.dot(U), sep='\n')

A:
[[11 65 20 71 19]
 [58 58 14 20 44]
 [26 92 91  1 79]
 [69 51 50 45 56]
 [73  0 59 86 54]]


L:
[[1.         0.         0.         0.         0.        ]
 [5.27272727 1.         0.         0.         0.        ]
 [2.36363636 0.21478873 1.         0.         0.        ]
 [6.27272727 1.25352113 0.62466926 1.         0.        ]
 [6.63636364 1.51760563 1.04087147 2.4601018  1.        ]]


U:
[[  11.           65.           20.           71.           19.        ]
 [   0.         -284.          -91.         -354.          -56.        ]
 [   0.            0.           62.54577465  -89.96478873   46.02816901]
 [   0.            0.            0.           99.94471655  -21.55519901]
 [   0.            0.            0.            0.           18.10449114]]


LU:
[[11.         65.         20.         71.         19.        ]
 [58.         58.72727273 14.45454545 20.36363636 44.18181818]
 [26.         92.63636364 90.27272727  1.81818182 78.90909091]
 [69.         51.72727273 50.45454545 45.363

In [21]:
A = np.random.random((5, 5))
print('A:', A, sep='\n', end='\n\n\n')
L, U = LU(A)
L = np.array(L)
print('L:', L, sep='\n', end='\n\n\n')
U = np.array(U)
print('U:', U, sep='\n', end='\n\n\n')
print('LU:',L.dot(U), sep='\n')

A:
[[0.58012025 0.10294686 0.52054478 0.13208916 0.19029768]
 [0.33523541 0.42767686 0.17143357 0.8902745  0.17094814]
 [0.13364254 0.09558094 0.63509292 0.7117266  0.6639077 ]
 [0.3738034  0.80519132 0.89252653 0.89096384 0.4322042 ]
 [0.88932918 0.71826528 0.8910102  0.02160115 0.85508882]]


L:
[[1.         0.         0.         0.         0.        ]
 [0.57787229 1.         0.         0.         0.        ]
 [0.23037041 0.19518637 1.         0.         0.        ]
 [0.64435503 2.00674532 1.51127387 1.         0.        ]
 [1.53300833 1.52218114 0.53650656 1.0513847  1.        ]]


U:
[[ 0.58012025  0.10294686  0.52054478  0.13208916  0.19029768]
 [ 0.          0.36818673 -0.12937482  0.81394384  0.06098038]
 [ 0.          0.          0.540427    0.52242642  0.60816621]
 [ 0.          0.          0.         -1.61705586 -0.73189286]
 [ 0.          0.          0.          0.          0.91375349]]


LU:
[[0.58012025 0.10294686 0.52054478 0.13208916 0.19029768]
 [0.33523541 0.42767686 0

## Ejercicio 3:
En este ejercicio te toca programar la descomposición $QR$ de una matriz $A$ La idea es usar el **Proceso de Gram-Schmidt**:

A partir de los vectores linealmente independientes $v_1 ,...,v_n$ se construye $u_1 ,...,u_n$ vectores normales. \begin{equation} u_{1} = v_{1}, \quad u_{k} = v_{k} - \sum {j = 1}^{k-1}\frac{\left \langle v{k}, u_{j} \right \rangle}{\left | u_{j} \right |^{2}} u_{j}, j = 2, \dots, k \end{equation} \begin{equation} e_{i} = \frac{u_{i}}{\left | u_{i} \right |} \end{equation}

**Recordatorio:** $‖u‖^2 =⟨u,u⟩$

**Paso 1:** Programa una función que calcule la proyección de un vector $u$ sobre otro vector $v$

In [22]:

def prod_punto(u, v):
    """
    Calcula el producto punto entre dos vectores u y v.
    """
    return sum([ui * vi for ui, vi in zip(u, v)])

def proyeccion(u, v):
    """
    Calcula la proyección de u sobre v.
    """
    uv = prod_punto(u, v)
    vv = prod_punto(v, v)
    coef = uv / vv
    for i in range(len(v)):
        v[i] = coef * v[i]
    return v


In [23]:
proyeccion([1.,1],[1.,0])

[1.0, 0.0]

**Paso 2:** Programa una función que calcule la descomposición $QR$ de una matriz $A$ y que regrese las matrices $Q$ y $R$.
\begin{equation} A =
\begin{bmatrix} a_{1}| a_{2}| \dots |a_{n}
\end{bmatrix} =
\begin{bmatrix} e_{1}| e_{2}| \dots |e_{n}
\end{bmatrix}
\begin{bmatrix} a_{1}\cdot e_{1} & a_{2}e_{1} & \cdots & a_{n}\cdot e_{1}\\
0 & a_{2}\cdot e_{2} & \cdots & a_{n}\cdot e_{2}\\
0 & 0 & \ddots & \vdots\\
0 & 0 & \cdots & a_{n}\cdot e_{n}
\end{bmatrix} \end{equation}

In [24]:
import numpy as np
from math import sqrt

def QR(M):
    """
    Realiza una factorización QR para una matriz M con columnas linealmente independientes.
    """
    # En este primer bloque buscamos los vectores columna que conformarán la matriz Q
    # ----------------------------------------------------------------------------------------------------
    A_Is = np.array(M.copy()).transpose()  # Transponemos la Matriz M para trabajar con sus vectores columna
    U_Is = [A_Is[0]]  # Aquí guardaremos los vectores u ortogonales

    # Usamos el proceso de Gram-Schmidt para obtener los vectores ortogonales a partir de A_Is
    # ----------------------------------------------------------------------------------------------------
    for i in range(1, len(A_Is)):
        res = sum([proyeccion(A_Is[i], k) for k in U_Is])
        U_Is.append(A_Is[i] - res)
    U_Is = np.array(U_Is)
    # ----------------------------------------------------------------------------------------------------

    # El siguiente bloque normaliza los vectores u
    E_Is = []
    for i in range(0, len(U_Is)):
        ei = U_Is[i] / sqrt(U_Is[i].dot(U_Is[i]))
        E_Is.append(ei)
    Q = np.array(E_Is).transpose()
    # ----------------------------------------------------------------------------------------------------

    # Finalmente, obtenemos la matriz R modificando una matriz triangular superior de nxm
    R = np.ones_like(M)
    R = np.triu(R)

    for i in range(len(R)):
        for j in range(len(R[0])):
            if j >= i:
                R[i][j] = A_Is[j].dot(E_Is[i])

    return Q, R  # Devolvemos ambas matrices


**Paso 3:**
Prueba tu funicón con diferentes matrices y corrobora que el producto de matrices $QR$ regresa la matriz original $A$ a la que se le encontró la factorización. Para llevar a cabo el producto de matrices $QR$ puedes utilziar la siguiente linea de código ``Q.dot(r)``

In [25]:
A = [[1,-2,4],[5,3,2],[2,-1,1]]
print('A:', np.array(A), sep='\n', end='\n\n\n')
Q, R = QR(A)
print('Q:', Q, sep='\n', end='\n\n\n')
print('R:', R, sep='\n', end='\n\n\n')
print('QR:', Q.dot(R), sep='\n')

A:
[[ 1 -2  4]
 [ 5  3  2]
 [ 2 -1  1]]


Q:
[[ 0.          0.70710678  0.90453403]
 [ 1.         -0.70710678  0.30151134]
 [ 0.          0.          0.30151134]]


R:
[[ 2  3  2]
 [ 0 -3  1]
 [ 0  0  4]]


QR:
[[ 0.         -2.12132034  4.32524292]
 [ 2.          5.12132034  2.4989386 ]
 [ 0.          0.          1.20604538]]
